# Tratar Dados

# Bibliotecas

In [27]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Tratamento de dados dicionário

## Importar Dicionários

In [28]:
def abrir_dics(path, sheet_name='dicionário pnad covid'):

    # Lê arquivo Excel no caminho especificado (path)
    prepdics = pd.read_excel(path, sheet_name=sheet_name, header=1, usecols='B:F')

    # Remove a coluna 'Quesito' e as primeiras 3 linhas
    prepdics = prepdics.iloc[3:].drop('Quesito', axis=1)

    # Renomeia as colunas
    prepdics.columns = ['variavel', 'questao', 'tipo', 'descricao']

    # Reset do índice
    prepdics = prepdics.reset_index(drop=True)

    return prepdics

dic09 = abrir_dics('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/Dicionario_PNAD_COVID_092020_20220621 (1).xls')
dic10 = abrir_dics('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/Dicionario_PNAD_COVID_102020_20220621 (1).xls')
dic11 = abrir_dics('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/Dicionario_PNAD_COVID_112020_20220621 (1).xls')

dics = [dic09, dic10, dic11]

Visualizar primeiras alterações feitas, dos 3 dicionarios

In [29]:
print("dic09:")
print(dic09.head(5))
print(dic09.shape)

print("\ndic10:")
print(dic10.head(5))
print(dic10.shape)

print("\ndic11:")
print(dic11.head(5))
print(dic11.shape)

dic09:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1      NaN                   NaN   12      Acre
2      NaN                   NaN   13  Amazonas
3      NaN                   NaN   14   Roraima
4      NaN                   NaN   15      Pará
(641, 4)

dic10:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1      NaN                   NaN   12      Acre
2      NaN                   NaN   13  Amazonas
3      NaN                   NaN   14   Roraima
4      NaN                   NaN   15      Pará
(641, 4)

dic11:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1      NaN                   NaN   12      Acre
2      NaN                   NaN   13  Amazonas
3      NaN                   NaN   14   Roraima
4      NaN                   NaN   15      Pará
(656, 4)


## Tirar valores null e preencher campos

In [30]:
def limpar_dicionarios(*dfs):
    # Lista para armazenar os dataframes limpos
    clean_dfs = []
    for df in dfs:
        # Preenche os valores NaN e faz uma cópia
        clean = df.ffill().copy()
        # Converte para minúsculas as colunas de string, exceto a coluna 'tipo'
        clean.apply(lambda x: x.str.lower() if x.name != 'tipo' and x.dtype == 'object' else x)
        # Substitui padrões nas colunas de string
        replaces = {'  ': ' ', r'\n': ' ', r'\r': ' ', r'/': ' ou ', r'%': '_por_cento'}
        clean.replace(replaces, regex=True, inplace=True)
        # Adiciona o dataframe limpo à lista
        clean_dfs.append(clean)
    return clean_dfs

clean_dfs = limpar_dicionarios(dic09, dic10, dic11)
dic09_clean, dic10_clean, dic11_clean = clean_dfs

dic_clean = [dic09_clean, dic10_clean, dic11_clean]

In [31]:
print("dic09:")
print(dic09_clean.head(5))
print(dic09_clean.shape)

print("\ndic10:")
print(dic10_clean.head(5))
print(dic10_clean.shape)

print("\ndic11:")
print(dic11_clean.head(5))
print(dic11_clean.shape)

dic09:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1       UF  Unidade da Federação   12      Acre
2       UF  Unidade da Federação   13  Amazonas
3       UF  Unidade da Federação   14   Roraima
4       UF  Unidade da Federação   15      Pará
(641, 4)

dic10:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1       UF  Unidade da Federação   12      Acre
2       UF  Unidade da Federação   13  Amazonas
3       UF  Unidade da Federação   14   Roraima
4       UF  Unidade da Federação   15      Pará
(641, 4)

dic11:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1       UF  Unidade da Federação   12      Acre
2       UF  Unidade da Federação   13  Amazonas
3       UF  Unidade da Federação   14   Roraima
4       UF  Unidade da Federação   15      Pará
(656, 4)


## Concatena os dicionários

In [6]:
def processar_dicionarios(dic_clean, drop_duplicates=True):
    # Concatena todos os dicionários
    df_dics = pd.concat(dic_clean)
    # Preenche os valores NaN
    df_dics = df_dics.ffill()
    # Remove duplicatas
    if drop_duplicates:
        df_dics = df_dics.drop_duplicates()
    # Remove duplicatas com base em 'variavel' e 'descricao'
    df_dics = df_dics.drop_duplicates(subset=['variavel', 'descricao'])

    return df_dics

# Usando a função
df_dics_processed = processar_dicionarios(dic_clean, drop_duplicates=True)
print(df_dics_processed.head(5), "\nshape:", df_dics_processed.shape)

  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1       UF  Unidade da Federação   12      Acre
2       UF  Unidade da Federação   13  Amazonas
3       UF  Unidade da Federação   14   Roraima
4       UF  Unidade da Federação   15      Pará 
shape: (650, 4)


## Criar excel do dicionário

In [7]:
# df_dics_processed.to_excel("/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/dicionario_tratado.xlsx")

## Visualizar as questões

In [8]:
def questions (df, variavel=None, key='tipo', value='questao'):
    if variavel is None:
        return df.set_index(key)[value].to_dict()
    else:
        return df.query(f'variavel == "{variavel}"').set_index(key)[value].to_dict()

# Visualizar todas as questões
questionsdic = questions(df_dics_processed, key='variavel', value='questao')
questionsdic

{'UF': 'Unidade da Federação',
 'CAPITAL': 'Capital',
 'RM_RIDE': 'Região Metropolitana e Região Administrativa Integrada de Desenvolvimento',
 'V1008': 'Número de seleção do domicílio',
 'V1012': 'Semana no mês',
 'V1013': 'Mês da pesquisa',
 'V1016': 'Número da entrevista no domicílio',
 'Estrato': 'Estrato',
 'UPA': 'UPA',
 'V1022': 'Situação do domicílio',
 'V1023': 'Tipo de área',
 'V1030': 'Projeção da população',
 'V1031': 'Peso do domicílio e das pessoas',
 'V1032': 'Peso do domicílio e das pessoas',
 'posest': 'Domínios de projeção',
 'A001': 'Número de ordem',
 'A001A': 'Condição no domicílio',
 'A001B1': 'Dia de nascimento',
 'A001B2': 'Mês de nascimento',
 'A001B3': 'Ano de nascimento',
 'A002': 'Idade do morador ',
 'A003': 'Sexo',
 'A004': 'Cor ou raça',
 'A005': 'Escolaridade',
 'A006': 'Frequenta escola',
 'A007': 'Na semana passada, _____ foram disponibilizadas atividades escolares para realizar em casa?',
 'A008': 'Na semana passada, em quantos dias _____ dedicou-se à

# Tratamento de dados PNAD COVID

3 meses para construção da solução

In [9]:
df09 = pd.read_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/PNAD_COVID_092020.csv')
df10 = pd.read_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/PNAD_COVID_102020.csv')
df11 = pd.read_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/PNAD_COVID_112020.csv')

covid_dfs = [df09, df10, df11]

for d in covid_dfs:
    print(f'shape: {d.shape}')

shape: (387298, 145)
shape: (380461, 145)
shape: (381438, 148)


## Compara as colunas

In [10]:
def comparar_colunas(df1, df2):
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)

    cols_only_in_df1 = cols1 - cols2
    cols_only_in_df2 = cols2 - cols1

    if cols_only_in_df1:
        print(f"Colunas presentes apenas em df1: {cols_only_in_df1}")
    if cols_only_in_df2:
        print(f"Colunas presentes apenas em df2: {cols_only_in_df2}")

# Comparar todos os pares de DataFrames
comparar_par = [(df09, df10), (df09, df11), (df10, df11)]

for df1, df2 in comparar_par:
    comparar_colunas(df1, df2)

Colunas presentes apenas em df2: {'A006B', 'A006A', 'A007A'}
Colunas presentes apenas em df2: {'A006B', 'A006A', 'A007A'}


## Concatenar

In [11]:
df_concat = pd.concat([df09, df10, df11])
df_concat.shape

(1149197, 148)

## Selecionar as colunas

In [12]:
# Lista de variáveis selecionadas
variaveis_selecionadas = ['UF', 'V1022', 'V1013', 'A002', 'A003', 'A004', 'A005',
                       'C001','F001', 'B0011', 'B0012', 'B0013', 'B0014',
                       'B0015','B0016', 'B0017', 'B0018', 'B0019', 'B00110',
                       'B00111', 'B00112', 'B00113', 'B002', 'B0041', 'B0042',
                       'B0043', 'B0044', 'B0045', 'B0046', 'B008', 'B009A',
                       'B009B', 'B009C', 'B009D', 'B009E', 'B009F', 'B0101',
                       'B0102', 'B0103', 'B0104', 'B0105', 'B0106', 'B011',
                       'C006', 'C007', 'C008', 'C009', 'C013', 'D0063',
                       'D0051', 'D0053', 'D0061', 'E001']

# Selecionando apenas as colunas desejadas
df_covid = df_concat[variaveis_selecionadas]

## Ajustado os dados usando do dicionário

In [13]:
def rename_codes(df, var_names):
    for var_name in var_names:
        # Crie um dicionário de códigos e descrições
        code_dict = df_dics_processed.query(f'variavel == "{var_name}"')[['tipo', 'descricao']].set_index('tipo')['descricao'].to_dict()

        # Renomeie a variável no DataFrame
        df.loc[:, var_name] = df[var_name].map(code_dict)

    return df


# Lista de variáveis para renomear
var_names_to_rename = ['UF', 'V1022', 'A003', 'A004', 'A005', 'F001', 'B0011', 'B0012', 'B0013', 'B0014',
                       'B0015','B0016', 'B0017', 'B0018', 'B0019', 'B00110',
                       'B00111', 'B00112', 'B00113', 'B008', 'B0101',
                       'B0102', 'B0103', 'B0104', 'B0105', 'B0106', 'B011', 'E001']


# Renomeie as variáveis no DataFrame
df_covid = rename_codes(df_covid, var_names_to_rename)

# Exiba as primeiras linhas do DataFrame
df_covid.head()

,UF,V1022,V1013,A002,A003,A004,A005,C001,F001,B0011,...,C006,C007,C008,C009,C013,D0063,D0051,D0053,D0061,E001
0,Rondônia,Urbana,9,36,Homem,Parda,Médio completo,1.0,Próprio - já pago,Não,...,2.0,4.0,48.0,48.0,NaN,NaN,1,1200.0,2,Não solicitou
1,Rondônia,Urbana,9,30,Mulher,Parda,Superior completo,1.0,Próprio - já pago,Não,...,2.0,7.0,36.0,36.0,NaN,NaN,1,1200.0,2,Não solicitou
2,Rondônia,Urbana,9,13,Homem,Parda,Fundamental incompleto,NaN,Próprio - já pago,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1200.0,2,Não solicitou
3,Rondônia,Urbana,9,11,Homem,Parda,Fundamental incompleto,NaN,Próprio - já pago,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1200.0,2,Não solicitou
4,Rondônia,Urbana,9,57,Mulher,Branca,Fundamental incompleto,2.0,Próprio - já pago,Não,...,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,2,Não solicitou


In [15]:
columns_to_map = {
    'D0051': {1.0: 'Sim', 2.0: 'Não'},
    'D0061': {1.0: 'Sim', 2.0: 'Não'},
    'C001': {1.0: 'Sim', 2.0: 'Não / Não aplicável'},
    'C006': {1.0: 'Sim', 2.0: 'Não / Não aplicável'},
    'C013': {1.0: 'Sim', 2.0: 'Não / Não aplicável'},
    'B002': {1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado / Não aplicável'},
    'B0041': {1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado / Não aplicável'},
    'B0042': {1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado / Não aplicável'},
    'B0043': {1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado / Não aplicável'},
    'B0044': {1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado / Não aplicável'},
    'B0045': {1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado / Não aplicável'},
    'B0046': {1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado / Não aplicável'},
    'B009A': {1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'},
    'B009C': {1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'},
    'B009E': {1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'},
    'B009B': {1.0: 'Positivo', 2.0: 'Negativo', 3.0: 'Inconclusivo', 4.0: 'Ainda não recebeu o resultado', 9.0: 'Ignorado / Não aplicável'},
    'B009D': {1.0: 'Positivo', 2.0: 'Negativo', 3.0: 'Inconclusivo', 4.0: 'Ainda não recebeu o resultado', 9.0: 'Ignorado / Não aplicável'},
    'B009F': {1.0: 'Positivo', 2.0: 'Negativo', 3.0: 'Inconclusivo', 4.0: 'Ainda não recebeu o resultado', 9.0: 'Ignorado / Não aplicável'}
}

for col, mapping in columns_to_map.items():
    df_covid[col] = df_covid[col].map(mapping).fillna('Ignorado / Não aplicável')

<ipython-input-15-b22a11dd9c13>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid[col] = df_covid[col].map(mapping).fillna('Ignorado / Não aplicável')


In [16]:
# Mapeamento para a coluna C007
c007_mapping = {
    1.0: 'Trabalhador doméstico (empregado doméstico, cuidados, babá)',
    2.0: 'Militar do exército, marinha ou aeronáutica',
    3.0: 'Policial militar ou bombeiro militar',
    4.0: 'Empregado do setor privado',
    5.0: 'Empregado do setor público (inclusive empresas de economia mista)',
    6.0: 'Empregador',
    7.0: 'Conta própria',
    8.0: 'Trabalhador familiar não remunerado em ajuda a membro do domicílio ou parente',
    9.0: 'Estava fora do mercado de trabalho (fazia apenas afazeres domésticos, cuidados de pessoas ou produção para próprio consumo)'
}

# Mapear a coluna C007
df_covid['C007'] = df_covid['C007'].map(c007_mapping).fillna('Não aplicável')

<ipython-input-16-fbdc2c505209>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['C007'] = df_covid['C007'].map(c007_mapping).fillna('Não aplicável')


## Verificar null

In [17]:
df_covid.isnull().sum()

UF              0
V1022           0
V1013           0
A002            0
A003            0
A004            0
A005            0
C001            0
F001            0
B0011           0
B0012           0
B0013           0
B0014           0
B0015           0
B0016           0
B0017           0
B0018           0
B0019           0
B00110          0
B00111          0
B00112          0
B00113          0
B002            0
B0041           0
B0042           0
B0043           0
B0044           0
B0045           0
B0046           0
B008            0
B009A           0
B009B           0
B009C           0
B009D           0
B009E           0
B009F           0
B0101           0
B0102           0
B0103           0
B0104           0
B0105           0
B0106           0
B011            0
C006            0
C007            0
C008       713200
C009       713200
C013            0
D0063     1130126
D0051           0
D0053      574880
D0061           0
E001            0
dtype: int64

## Preencher os NaN

In [18]:
# NaN to 0
df_covid = df_covid.fillna(0)
df_covid.head()

,UF,V1022,V1013,A002,A003,A004,A005,C001,F001,B0011,...,C006,C007,C008,C009,C013,D0063,D0051,D0053,D0061,E001
0,Rondônia,Urbana,9,36,Homem,Parda,Médio completo,Sim,Próprio - já pago,Não,...,Não / Não aplicável,Empregado do setor privado,48.0,48.0,Ignorado / Não aplicável,0.0,Sim,1200.0,Não,Não solicitou
1,Rondônia,Urbana,9,30,Mulher,Parda,Superior completo,Sim,Próprio - já pago,Não,...,Não / Não aplicável,Conta própria,36.0,36.0,Ignorado / Não aplicável,0.0,Sim,1200.0,Não,Não solicitou
2,Rondônia,Urbana,9,13,Homem,Parda,Fundamental incompleto,Ignorado / Não aplicável,Próprio - já pago,Não,...,Ignorado / Não aplicável,Não aplicável,0.0,0.0,Ignorado / Não aplicável,0.0,Sim,1200.0,Não,Não solicitou
3,Rondônia,Urbana,9,11,Homem,Parda,Fundamental incompleto,Ignorado / Não aplicável,Próprio - já pago,Não,...,Ignorado / Não aplicável,Não aplicável,0.0,0.0,Ignorado / Não aplicável,0.0,Sim,1200.0,Não,Não solicitou
4,Rondônia,Urbana,9,57,Mulher,Branca,Fundamental incompleto,Não / Não aplicável,Próprio - já pago,Não,...,Ignorado / Não aplicável,Não aplicável,0.0,0.0,Ignorado / Não aplicável,0.0,Não,0.0,Não,Não solicitou


In [19]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1149197 entries, 0 to 381437
Data columns (total 53 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   UF      1149197 non-null  object 
 1   V1022   1149197 non-null  object 
 2   V1013   1149197 non-null  int64  
 3   A002    1149197 non-null  int64  
 4   A003    1149197 non-null  object 
 5   A004    1149197 non-null  object 
 6   A005    1149197 non-null  object 
 7   C001    1149197 non-null  object 
 8   F001    1149197 non-null  object 
 9   B0011   1149197 non-null  object 
 10  B0012   1149197 non-null  object 
 11  B0013   1149197 non-null  object 
 12  B0014   1149197 non-null  object 
 13  B0015   1149197 non-null  object 
 14  B0016   1149197 non-null  object 
 15  B0017   1149197 non-null  object 
 16  B0018   1149197 non-null  object 
 17  B0019   1149197 non-null  object 
 18  B00110  1149197 non-null  object 
 19  B00111  1149197 non-null  object 
 20  B00112  1149197 non-null  obje

## Renomear as colunas

In [20]:
# Dicionário com os novos nomes das variáveis
df_covid = df_covid.rename(columns={
    'UF': 'uf',
    'V1022': 'situacao_domiciliar',
    'V1013': 'mes',
    'A002': 'idade',
    'A003': 'sexo',
    'A004': 'cor',
    'A005': 'escolaridade',
    'C001': 'trabalhou_semana_passada',
    'F001': 'tipo_moradia',
    'B0011': 'febre',
    'B0012': 'tosse',
    'B0013': 'dor_garganta',
    'B0014': 'dificuldade_respirar',
    'B0015': 'dor_cabeca',
    'B0016': 'dor_peito',
    'B0017': 'nausea',
    'B0018': 'nariz_entupido',
    'B0019': 'fadiga',
    'B00110': 'dor_olhos',
    'B00111': 'perda_olfato_paladar',
    'B00112': 'dor_muscular',
    'B00113': 'diarreia',
    'B002':  'ida_unidade_saude',
    'B0041': 'posto_publico',
    'B0042': 'pronto_socorro_publico',
    'B0043': 'hospital_publico',
    'B0044': 'ambulatorio_privado',
    'B0045': 'pronto_socorro_privado',
    'B0046': 'hospital_privado',
    'B008': 'fez_teste',
    'B009A': 'teste_swab',
    'B009B': 'resultado_swab',
    'B009C': 'teste_sangue_dedo',
    'B009D': 'resultado_sangue_dedo',
    'B009E': 'teste_sangue_coleta',
    'B009F': 'resultado_sangue_coleta',
    'B0101': 'diagnostico_diabetes',
    'B0102': 'diagnostico_hipertensao',
    'B0103': 'diagnostico_doenca_pulmao',
    'B0104': 'diagnostico_doenca_cardiaca',
    'B0105': 'diagnostico_depressao',
    'B0106': 'diagnostico_cancer',
    'B011': 'contato_restricao',
    'C006': 'mais_de_um_trabalho',
    'C007': 'tipo_trabalho',
    'C008': 'horas_trabalho_normal',
    'C009': 'horas_trabalho_semana_passada',
    'C013': 'trabalho_remoto_semana_passada',
    'D0063': 'valor_recebido_seguro_desemprego',
    'D0051': 'auxilios_emergenciais_coronavirus',
    'D0053': 'valor_recebido_auxilios_emergenciais',
    'D0061': 'seguro_desemprego',
    'E001': 'solicitou_emprestimo'
})

df_covid.head()

,uf,situacao_domiciliar,mes,idade,sexo,cor,escolaridade,trabalhou_semana_passada,tipo_moradia,febre,...,mais_de_um_trabalho,tipo_trabalho,horas_trabalho_normal,horas_trabalho_semana_passada,trabalho_remoto_semana_passada,valor_recebido_seguro_desemprego,auxilios_emergenciais_coronavirus,valor_recebido_auxilios_emergenciais,seguro_desemprego,solicitou_emprestimo
0,Rondônia,Urbana,9,36,Homem,Parda,Médio completo,Sim,Próprio - já pago,Não,...,Não / Não aplicável,Empregado do setor privado,48.0,48.0,Ignorado / Não aplicável,0.0,Sim,1200.0,Não,Não solicitou
1,Rondônia,Urbana,9,30,Mulher,Parda,Superior completo,Sim,Próprio - já pago,Não,...,Não / Não aplicável,Conta própria,36.0,36.0,Ignorado / Não aplicável,0.0,Sim,1200.0,Não,Não solicitou
2,Rondônia,Urbana,9,13,Homem,Parda,Fundamental incompleto,Ignorado / Não aplicável,Próprio - já pago,Não,...,Ignorado / Não aplicável,Não aplicável,0.0,0.0,Ignorado / Não aplicável,0.0,Sim,1200.0,Não,Não solicitou
3,Rondônia,Urbana,9,11,Homem,Parda,Fundamental incompleto,Ignorado / Não aplicável,Próprio - já pago,Não,...,Ignorado / Não aplicável,Não aplicável,0.0,0.0,Ignorado / Não aplicável,0.0,Sim,1200.0,Não,Não solicitou
4,Rondônia,Urbana,9,57,Mulher,Branca,Fundamental incompleto,Não / Não aplicável,Próprio - já pago,Não,...,Ignorado / Não aplicável,Não aplicável,0.0,0.0,Ignorado / Não aplicável,0.0,Não,0.0,Não,Não solicitou


## Exporta dados tratados

In [22]:
df_covid.to_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/df_covid.csv')